# Segmentation
The purpose of this code is to perform the segmentation of the brain tumor for subsequent feature extraction. The segmentation is the second step of this multi-step process, with the first step being image preprocessing, the second being segmentation, and the third being classification. This code is sourced from user [Issam28](https://github.com/Issam28/Brain-tumor-segmentation) on GitHub, who uses a CNN. This notebook will serve as a synthesis of all the different files in the repository with a few needed tweaks.


We must also need to install a package to read NiFTI files.

In [2]:
# !pip install --quiet nibabel
import nibabel as nib

## Evaluation Metrics
This code will create evaluation metrics that will assess how accurately the model can automatically segment the tumor. We will first add the necessary import statements.

In [3]:
import numpy as np
from scipy import ndimage

We will then create the method used to calculate the Dice score of two 3D volumes. The Dice score is a statistic used to assess the performance of image segmentation methods. The higher the Dice score, the more accurate the segmentation model is.

In [95]:
def binary_dice3d(s, g):
    # g=np.transpose(g,(2, 1, 0))
    num = np.sum(np.multiply(s, g))
    denom = s.sum() + g.sum()
    if denom == 0:
        return 1
    else:
        return 2.0 * num / denom

The next method we will define will calculate the sensitivity and specificity metrics, or the false negative rate and false positive rates, respectively.



In [96]:
def sensitivity(seg, ground):
    # ground =np.transpose(ground,(2, 1, 0))
    num = np.sum(np.multiply(ground, seg))
    denom = np.sum(ground)
    if denom == 0:
        return 1
    else:
        return num / denom

    
def specificity(seg, ground):
    # ground =np.transpose(ground,(2, 1, 0))
    num = np.sum(np.multiply(ground == 0, seg == 0))
    denom = np.sum(ground == 0)
    if denom == 0:
        return 1
    else:
        return num / denom

The next method will create a border for any given 3D image.

In [97]:
def border_map(binary_img, n):
    binary_map = np.asarray(binary_img, dtype = np.uint8)
    neigh = n
    west = ndimage.shift(binary_map, [-1, 0, 0], order = 0)
    east = ndimage.shift(binary_map, [1, 0, 0], order = 0)
    north = ndimage.shift(binary_map, [0, 1, 0], order = 0)
    south = ndimage.shift(binary_map, [0, -1, 0], order = 0)
    top = ndimage.shift(binary_map, [0, 0, 1], order = 0)
    bottom = ndimage.shift(binary_map, [0, 0, -1], order = 0)
    cumulative = west + east + north + south + top + bottom
    border = ((cumulative < 6) * binary_map) == 1
    return border

The method `border_distance` will create a map of the distances from the borders of the segmentation and the reference image, as well as the border maps themselves. The reference image, or `ref` is the correctly labeled MRI image of the tumor, while the segmentation, or `seg` is the model's attempt at segmenting the tumor.

In [98]:
def border_distance(ref, seg):
    neigh = 8
    border_ref = border_map(seg, neigh)
    border_seg = border_map(seg, neigh)
    oppose_ref = 1 - ref
    oppose_seg = 1 - seg
    # euclidean distance transform
    distance_ref = ndimage.distance_transform_edt(oppose_ref)
    distance_seg = ndimage.distance_transform_edt(oppose_seg)
    distance_border_seg = border_ref * distance_seg
    distance_border_ref = border_seg * distance_ref
    return distance_border_ref, distance_border_seg

This next method will calculate the average symmetric distance and subsequently the Hausdorff distance between a segmentation and a reference image. The Hausdorff distance measures how far two subsets of a metric space are from each other. The average symmetric distance essentially calculates how accurate the segmentation of the image is compared to the reference. A perfect segmentation has an average symmetric distance value of 0.

In [99]:
def Hausdorff_distance(ref, seg):
    ref_border_dist, seg_border_dist = border_distance(ref, seg)
    hausdorff_distance = np.max([np.max(ref_border_dist), 
                               np.max(seg_border_dist)])
    return hausdorff_distance

The next three methods will compute the Dice score for the whole tumor, the enhancing region, and the core region, respectively. The enhancing region of a tumor is defined as the region that continues to spread and grow, and the core region is the localized center of the tumor.

In [100]:
# whole region
def DSC_whole(pred, orig_label):
    return binary_dice3d(pred > 0, orig_label > 0)


# enhancing region
def DSC_en(pred, orig_label):
    return binary_dice3d(pred == 4, orig_label == 4)


# core region
def DSC_core(pred, orig_label):
    seg_ = np.copy(pred)
    ground_ = np.copy(orig_label)
    seg_[seg_ == 2] = 0
    ground_[ground_ == 2] = 0
    return binary_dice3d(seg_ > 0, ground_ > 0)

We will repeat this process for the specificity metric (false positive rate).

In [101]:
# whole region
def specificity_whole(seg, ground):
    return specificity(seg > 0, ground > 0)


# enhancing region
def specificity_en(seg, ground):
    return specificity(seg == 4, ground == 4)


# core region
def specificity_core(seg, ground):
    seg_ = np.copy(seg)
    ground_ = np.copy(ground)
    seg_[seg_ == 2] = 0
    ground_[ground_ == 2] = 0
    return specificity(seg_ > 0, ground_ > 0)

And also the sensitivity metric.

In [102]:
def sensitivity_whole(seg,ground):
    return sensitivity(seg>0,ground>0)

def sensitivity_en(seg,ground):
    return sensitivity(seg==4,ground==4)

def sensitivity_core(seg,ground):
    seg_=np.copy(seg)
    ground_=np.copy(ground)
    seg_[seg_==2]=0
    ground_[ground_==2]=0
    return sensitivity(seg_>0,ground_>0)

Lastly, we will finish with finding the Hausdorff distance in the same manner with three methods.

In [103]:
# whole region
def hausdorff_whole(seg, ground):
    return Hausdorff_distance(seg == 0, ground == 0)


# enhancing region
def hausdorff_en(seg, ground):
    return Hausdorff_distance(seg != 4, ground != 4)


# core region
def hausdorff_core(seg, ground):
    seg_ = np.copy(seg)
    ground_ = np.copy(ground)
    seg_[seg_== 2] = 0
    ground_[ground_ == 2] = 0
    return Hausdorff_distance(seg_ == 0, ground_ == 0)

## Loss Functions
This code will define the loss functions for the model itself. We will be using the Tensorflow backend, which is needed for low-level operations such as tensor products and convolutions, which we will need to import.

In [12]:
# !pip install keras
# !pip install -U tensorflow
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


The next four methods, the Dice score will be computed on two tensors (often thought of as a generalized matrix that can take on any numerical dimension), the whole tumor, the enhancing region, and the core region, respectively. Note that this is varies from the methods defined above in that the above methods assess the model after prediction, while these methods are defined and used during the actual running of the model.

In [13]:
# two tensors
def dice(y_true, y_pred):
    #computes the dice score on two tensors
    sum_p=K.sum(y_pred,axis=0)
    sum_r=K.sum(y_true,axis=0)
    sum_pr=K.sum(y_true * y_pred,axis=0)
    dice_numerator =2*sum_pr
    dice_denominator =sum_r+sum_p
    dice_score =(dice_numerator+K.epsilon() )/(dice_denominator+K.epsilon())
    return dice_score


# whole tumor
def dice_whole_metric(y_true, y_pred):
    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))
    y_whole=K.sum(y_true_f[:,1:],axis=1)
    p_whole=K.sum(y_pred_f[:,1:],axis=1)
    dice_whole=dice(y_whole,p_whole)
    return dice_whole


# enhancing region
def dice_en_metric(y_true, y_pred):
    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))
    y_enh=y_true_f[:,-1]
    p_enh=y_pred_f[:,-1]
    dice_en=dice(y_enh,p_enh)
    return dice_en


# core region
def dice_core_metric(y_true, y_pred):
    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))

    #workaround for tf
    y_core=K.sum(tf.gather(y_true_f, [1,3],axis =1),axis=1)
    p_core=K.sum(tf.gather(y_pred_f, [1,3],axis =1),axis=1)

    #y_core=K.sum(y_true_f[:,[1,3]],axis=1)
    #p_core=K.sum(y_pred_f[:,[1,3]],axis=1)
    dice_core=dice(y_core,p_core)
    return dice_core

We will also need to define a weighted logarithmic loss function, which will have three parts. The first part will scale predictions so that the class predictions of each sample sum to 1. Then, we will clip to prevent `NaN` and `Inf` values. Lastly, weights will be assigned in the following order: normal, necrotic, edema, and enhancing. Necrotic brain tissue is unhealthily soft brain tissue, and edema is the accumulation of excess fluid in fluid compartments.

In [14]:
def weighted_log_loss(y_true, y_pred):
    # scale predictions
    y_pred /= K.sum(y_pred, axis = 1, keepdims = True)

    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

    # weights are assigned in this order: normal, necrotic, edema, enchancing
    weights = np.array([1, 5, 2, 4])
    weights = K.variable(weights)
    loss = y_true * K.log(y_pred) * weights
    loss = K.mean(-K.sum(loss, -1))
    return loss

The next method will compute the sum of two loss functions: the generalized dice loss function and the weighted cross-entropy function, which was the previously defined `weighted_log_loss` function. This will serve as our primary loss function for the model.

In [15]:
def gen_dice_loss(y_true, y_pred):
    y_true_f = K.reshape(y_true, shape = (-1, 4))
    y_pred_f = K.reshape(y_pred, shape = (-1, 4))
    sum_p = K.sum(y_pred_f, axis = -2)
    sum_r = K.sum(y_true_f, axis = -2)
    sum_pr = K.sum(y_true_f * y_pred_f, axis = -2)
    weights = K.pow(K.square(sum_r) + K.epsilon(), -1)
    generalized_dice_numerator = 2 * K.sum(weights * sum_pr)
    generalized_dice_denominator = K.sum(weights * (sum_r + sum_p))
    generalized_dice_score = generalized_dice_numerator / generalized_dice_denominator
    GDL = 1 - generalized_dice_score
    del sum_p, sum_r, sum_pr, weights
    return GDL + weighted_log_loss(y_true, y_pred)

## Model Creation
Code that creates the model class, which is a convolutional neural network. We will need multiple functions and modules from the Keras library, which we will import.

In [16]:
from keras.models import Model, load_model
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dropout, GaussianNoise, Input, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2DTranspose, UpSampling2D, concatenate, add
from keras.optimizers import SGD

Before creating the model, we must also set the image data format to `channels_last`. This parameter affects how the Tensorflow backend treats the data dimensions when working with convolutions in the model. This is the way that the data is stored as convolutions are performed. 

In [17]:
K.set_image_data_format("channels_last")

Now, we will actually create the model class, called U-Net. The U-Net model will have an encoding unit as well as a decoding unit, a batch normalization layer, a `PReLU` activation layer in the hidden layers, a 2D convolutional layer, and a `softmax`activation layer as the output.

In [18]:
class Unet_model(object):

    
    # constructor
    def __init__(self, img_shape, load_model_weights = None):
        self.img_shape = img_shape
        self.load_model_weights = load_model_weights
        self.model = self.compile_unet()
  

    # compile the u-net model
    def compile_unet(self):
        i = Input(shape = self.img_shape)
        # add gaussian noise to the first layer to combat overfitting
        i_ = GaussianNoise(0.01)(i)
        i_ = Conv2D(64, 2, padding = 'same', data_format = 'channels_last')(i_)
        out = self.unet(inputs = i_)
        model = Model(input = i, output = out)
    
        sgd = SGD(lr = 0.08, momentum = 0.9, decay = 5e-6, nesterov = False) ## Look at this documentation for saving?
        model.compile(loss = gen_dice_loss, optimizer = sgd, 
                      metrics = [dice_whole_metric, dice_core_metric, dice_en_metric])
        # load weights if set for prediction
        if self.load_model_weights is not None:
            model.load_weights(self.load_model_weights)
        return model
  

    # the actual u-net architecture
    def unet(self, inputs, nb_classes = 4, start_ch = 64, depth = 3, inc_rate = 2.,
          activation = 'relu', dropout = 0.0, batchnorm = True, upconv = True, format_='channels_last'):
        o = self.level_block(inputs, start_ch, depth, inc_rate, activation, dropout, batchnorm, upconv, format_)
        o = BatchNormalization()(o)
        o = PReLU(shared_axes=[1, 2])(o)
        o = Conv2D(nb_classes, 1, padding = 'same', data_format = format_)(o)
        o = Activation('softmax')(o)
        return o
  

    def level_block(self,m, dim, depth, inc, acti, do, bn, up,format_="channels_last"):
        if depth > 0:
            n = self.res_block_enc(m,0.0,dim,acti, bn,format_)
            # print("n after res_block_enc", n.shape)
            #using strided 2D conv for donwsampling
            m = Conv2D(int(inc*dim), 2,strides=2, padding='same',data_format = format_)(n)
            # print("m after conv2d", m.shape)
            m = self.level_block(m,int(inc*dim), depth-1, inc, acti, do, bn, up )
            if up:
                m = UpSampling2D(size=(2, 2),data_format = format_)(m)
                # print("m after upsampling", m.shape)
                m = Conv2D(dim, 2, padding='same',data_format = format_)(m)
                # print("m after conv2d", m.shape)
            else:
                m = Conv2DTranspose(dim, 3, strides=2,padding='same',data_format = format_)(m)
                # print("m after conv2dtranspose", m.shape)
            n=concatenate([n,m])
            #the decoding path
            m = self.res_block_dec(n, 0.0,dim, acti, bn, format_)
        else:
            m = self.res_block_enc(m, 0.0,dim, acti, bn, format_)
        return m

    
    def res_block_enc(self,m, drpout,dim,acti, bn,format_="channels_last"):
        
        """
        the encoding unit which a residual block
        """
        n = BatchNormalization()(m) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format = format_)(n)
                
        n = BatchNormalization()(n) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format =format_ )(n)

        n=add([m,n]) 
        
        return  n 
  

    def res_block_dec(self,m, drpout,dim,acti, bn,format_="channels_last"):

        """
        the decoding unit which a residual block
        """
         
        n = BatchNormalization()(m) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format = format_)(n)

        n = BatchNormalization()(n) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format =format_ )(n)
        
        Save = Conv2D(dim, 1, padding='same',data_format = format_,use_bias=False)(m) 
        n=add([Save,n]) 
        
        return  n   

## Patch Extraction
This code will sample patches of the MRI images that will be used for training. This patch extraction greatly reduces computational time. We will begin by importing the necessary modules and methods.

In [19]:
!pip install SimpleITK

In [19]:
import random
from skimage import io
import numpy as np
from glob import glob
import SimpleITK as sitk
from keras.utils import np_utils

We will create a Pipeline class for the process of extracting random patches of the 3D images for training the model. The Pipeline class will also contain methods to normalize the batches. 

In [20]:
class Pipeline(object):
  

    # constructor
    def __init__(self, list_train, Normalize = True):
        self.scans_train = list_train
        self.train_im = self.read_scans(Normalize)
  

    '''
    input: unnormalized slice
    output: normalized clipped slice
    '''
    def _normalize(self, slice):
        b = np.percentile(slice, 99)
        t = np.percentile(slice, 1)
        slice = np.clip(slice, t, b)
        image_nonzero = slice[np.nonzero(slice)]
        if np.std(slice) == 0 or np.std(image_nonzero) == 0:
            return slice
        else:
            tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
            # since the range of intensities is between 0 and 5000, the min in the
            # normalized slice corresponds to 0 intensity in unnormalized slice
            # the min is replaced with -9 just to keep track of 0 intensities so that
            # we can discard those intensities afterwards when sampling random patches
            tmp[tmp == tmp.min()] = -9
            return tmp
  

    '''
    normalizes each slice
    subtracts mean and div by std dev for each slice
    clips top and bottom one percent of pixel intensities
    '''
    def norm_slices(self, slice_not):
        normed_slices = np.zeros((5, 155, 240, 240)).astype(np.float32)
        for slice_ix in range(4):
            normed_slices[slice_ix] = slice_not[slice_ix]
            for mode_ix in range(155):
                normed_slices[slice_ix][mode_ix] = self._normalize(slice_not[slice_ix][mode_ix])
        normed_slices[-1] = slice_not[-1]

        return normed_slices

    
    '''
    concatenate two parts in one dataset
    this can be avoided if there is enough RAM
    '''
    def concatenate():
        Y_labels_2 = np.load("y_dataset_second_part.npy").astype(np.uint8)
        X_patches_2 = np.load("x_dataset_second_part.npy").astype(np.float32)
        Y_labels_1 = np.load("y_dataset_first_part.npy").astype(np.uint8)
        X_patches_1 = np.load("x_dataset_first_part.npy").astype(np.float32)

        # concatenate both parts
        X_patches = np.concatenate((X_patches_1, X_patches_2), axis = 0)
        Y_labels = np.concatenate((Y_labels_1, Y_labels_2), axis = 0)
        del Y_labels_2, X_patches_2, Y_labels_1, X_patches_1

        # shuffle the entire dataset
        shuffle = list(zip(X_patches, Y_labels))
        np.random.seed(138)
        np.random.shuffle(shuffle)
        X_patches = np.array([shuffle[i][0] for i in range(len(shuffle))])
        Y_labels = np.array([shuffle[i][1] for i in range(len(shuffle))])
        del shuffle

        np.save("x_training", X_patches.astype(np.float32))
        np.save("y_training", Y_labels.astype(np.uint8))
  

    def read_scans(self, Normalize):
        print("reading scans...")
        train_images = []
        count = 0
        for i in range(len(self.scans_train)):
            if i % 10 == 0:
                print('iteration [{}]'.format(i))
            folder_name = self.scans_train[i][self.scans_train[i].rfind('/')+1:]
            flair = glob(self.scans_train[i] + '/*_flair.nii.gz')
            t2 = glob(self.scans_train[i] + '/*_t2.nii.gz')
            gt = glob(self.scans_train[i] + '/*_seg.nii.gz')
            t1 = glob(self.scans_train[i] + '/*_t1.nii.gz')
            t1c = glob(self.scans_train[i] + '/*_t1ce.nii.gz')

            if (len(flair) + len(t2) + len(gt) + len(t1) + len(t1c)) < 5:
                # print("there's a problem here. problem lies in this patient:", folder_name)
                count = count + 1
                continue
            scans = [flair[0], t1[0], t1c[0], t2[0], gt[0]]

            # read a volume composed of 5 modalities
            # print(scans)
            tmp = []
            for k in range(len(scans)):
                img = np.array(nib.load(scans[k]).dataobj)
                tmp.append(img)
            
            tmp = np.transpose(tmp, (0, 3, 2, 1))
            # normalize each slice
            if Normalize == True:
                tmp = self.norm_slices(tmp)

            train_images.append(tmp)
            del tmp
        print("finished reading")
        print(count, "invalid folders/patients")
        return np.array(train_images)
    
    
    ''' 
    input:
    num_patches: the total number of sampled patches
    d: this corresponds to the number of channels (1 modality)
    h: height of the patch
    w: width of the patch

    output:
    patches: np array containing the randomly sampled patches
    labels: np array containing the corresponding target patches
    '''
    def sample_patches_randomly(self, num_patches, d, h, w):
        print("begin sample_patches_randomly function")
        patches = []
        labels = []
        count = 0

        # swap axes to make axis 0 represent the modality and axis 1 represent the slice
        # take the ground truth
        gt_im = np.swapaxes(self.train_im, 0, 1)[4]

        # take the flair image as a mask
        msk = np.swapaxes(self.train_im, 0, 1)[0]
        # save the shape of the ground truth to use it afterwards
        tmp_shp = gt_im.shape

        # reshape the mask and the ground truth to a 1D array
        gt_im = gt_im.reshape(-1).astype(np.uint8)
        msk = msk.reshape(-1).astype(np.float32)

        # maintain list of 1D indices while discarding 0 intensities
        indices = np.squeeze(np.argwhere((msk != -9.0) & (msk != 0.0)))
        del msk

        # shuffle the list of indices of the class
        np.random.shuffle(indices)

        # reshape gt_im
        gt_im = gt_im.reshape(tmp_shp)

        # a loop to sample the patches from the images
        i = 0
        pix = len(indices)
        print("sampling patches")
        while (count < num_patches) and (pix > i):
            # randomly choose an index
            ind = indices[i]
            i += 1
            # reshape ind to 3D index
            ind = np.unravel_index(ind, tmp_shp)
            # get the patient and the slice id (might need to rework)
            patient_id = ind[0]
            slice_idx = ind[1]
            p = ind[2:]
            # construct the patch by defining the coordinates
            p_y = (p[0] - (h)/2, p[0] + (h)/2)
            p_x = (p[1] - (w)/2, p[1] + (w)/2)
            p_x = list(map(int, p_x))
            p_y = list(map(int, p_y))

            # take patches from the modalities
            tmp = self.train_im[patient_id][0:4, slice_idx, p_y[0]:p_y[1], p_x[0]:p_x[1]]
            # take the corresponding label patch (might need to rework later)
            lbl = gt_im[patient_id, slice_idx, p_y[0]:p_y[1], p_x[0]:p_x[1]]

            # keep only patches that have the desired size
            if tmp.shape != (d, h, w):
                continue
            patches.append(tmp)
            labels.append(lbl)
            count += 1
        patches = np.array(patches)
        labels = np.array(labels)
        return patches, labels

## Training
Now, we will train the model on the training dataset. We will add the necessary import statements.

In [21]:
import json
from keras.models import model_from_json, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler

We will also be creating an `SGDLearningRateTracker` class. As the learning rate (`lr`) and `decay` values of the model change, this class will notify us of those changes in the model.

In [22]:
class SGDLearningRateTracker(Callback):
    def on_epoch_begin(self, epoch, logs = {}):
        optimizer = self.model.optimizer
        lr = K.get_value(optimizer.lr)
        decay = K.get_value(optimizer.decay)
        lr = lr / 10
        decay = decay * 10
        K.set_value(optimizer.lr, lr)
        K.set_value(optimizer.decay, decay)
        print("lr changed to:", lr)
        print("decay changed to:", decay)

We will create a `Training` class, primarily to synthesize all of the methods into one class to reduce clunky code.

In [23]:
class Training(object):
    
            
    '''
    load a model
    input: string 'model_name': filepath to model and weights, not including extension
    output: model with loaded weights. can fit on model using loaded_model = True in fit_model method
    '''
    def load_model(self, model_name):
        '''
        Load a model
        INPUT  (1) string 'model_name': filepath to model and weights, not including extension
        OUTPUT: Model with loaded weights. can fit on model using loaded_model=True in fit_model method
        '''
        print ('Loading model {}'.format(model_name))
        model_toload = '/Users/kaitlinylim/Documents/tumorproj/models/{}.json'.format(model_name)
        weights = '/Users/kaitlinylim/Documents/tumorproj/weights/{}.hdf5'.format(model_name)
        with open(model_toload) as f:
            loaded_model_json = f.read()
        model_comp = model_from_json(loaded_model_json)
        model_comp.load_weights(weights)
        print ('model loaded.')
        model_comp.compile(loss = gen_dice_loss, optimizer = SGD(lr = 0.08, momentum = 0.9, decay = 5e-6, nesterov = False), 
                           metrics = [dice_whole_metric, dice_core_metric, dice_en_metric])
        print('model compiled')
        self.model = model_comp
        return model_comp

    
    # constructor
    def __init__(self, batch_size, nb_epoch, load_model_resume_training = None):
        self.batch_size = batch_size
        self.nb_epoch = nb_epoch

        #loading model from path to resume previous training without recompiling the whole model
        if load_model_resume_training is not None:
            # self.model = self.load_model(load_model_resume_training,custom_objects={'gen_dice_loss': gen_dice_loss,'dice_whole_metric':dice_whole_metric,'dice_core_metric':dice_core_metric,'dice_en_metric':dice_en_metric})
            self.model = self.load_model(load_model_resume_training)
            print("pre-trained model loaded!")
        else:
            unet = Unet_model(img_shape = (128, 128, 4))
            self.model = unet.model
            print("u-net cnn compiled!")

    
    def fit_unet(self, X33_train, Y_train, X_patches_valid = None, Y_labels_valid = None):
        train_generator = self.img_msk_gen(X33_train, Y_train, 9999)
        checkpointer = ModelCheckpoint(filepath = "/Users/kaitlinylim/Documents/tumorproj/checkpoints/ResUnet.{epoch:02d}_{val_loss:.3f}.hdf5",
                                      verbose = 1)
        self.model.fit_generator(train_generator, steps_per_epoch = len(X33_train)//self.batch_size,
                                epochs = self.nb_epoch, validation_data = (X_patches_valid, Y_labels_valid),
                                verbose = 1, callbacks = [checkpointer, SGDLearningRateTracker()])

        
    '''
    a custom generator that performs data augmentation on both patches and their corresponding masks
    '''
    def img_msk_gen(self, X33_train, Y_train, seed):
        datagen = ImageDataGenerator(horizontal_flip = True, data_format = 'channels_last')
        datagen_msk = ImageDataGenerator(horizontal_flip = True, data_format = 'channels_last')
        image_generator = datagen.flow(X33_train, batch_size = 4, seed = seed)
        y_generator = datagen_msk.flow(Y_train, batch_size = 4, seed = seed)
        while True:
            yield(image_generator.next(), y_generator.next())

            
    '''
    input: string 'model_name': path where to save model and weights without extension
    saves current model as json and weights as h5df file
    '''
    def save_model(self, model_name):
        model_tosave = '{}.json'.format(model_name)
        json_string = self.model.to_json()
        with open('/Users/kaitlinylim/Documents/tumorproj/models/'+model_tosave, 'w') as json_file:
            json_file.write(json_string)
            weights = '{}.hdf5'.format(model_name)
            self.model.save_weights('/Users/kaitlinylim/Documents/tumorproj/weights/'+weights)
        print ('model saved')

## Prediction
This is the code for the actual tumor segmentation. We will begin with import statements.

In [24]:
import os

As with the `Training` class, we will also create a `Prediction` class to streamline the predicting process.

In [172]:
class Prediction(object):

    
    # constructor
    def __init__(self, batch_size_test, load_model_path):
        self.batch_size_test = batch_size_test
        unet = Unet_model(img_shape=(240, 240, 4), load_model_weights = load_model_path)
        self.model = unet.model
        print("u-net cnn model compiled\n")
  

    '''
    segment the input volume
    input: (1) str 'filepath_image': filepath of the volume to predict
     (2) bool 'show'
    output: (1) np array of the predicted volume
      (2) np array of the corresponding ground truth
    '''
    def predict_volume(self, filepath_image, show):
        
        '''
        segment the input volume
        INPUT   (1) str 'filepath_image': filepath of the volume to predict 
                (2) bool 'show': True to ,
        OUTPUt  (1) np array of the predicted volume
                (2) np array of the corresping ground truth
        '''
        
        #read the volume
        flair = glob(filepath_image + '/*_flair.nii.gz')
        t2 = glob(filepath_image + '/*_t2.nii.gz')
        gt = glob(filepath_image + '/*_seg.nii.gz')
        t1s = glob(filepath_image + '/*_t1.nii.gz')
        t1c = glob(filepath_image + '/*_t1ce.nii.gz')
        if (len(flair)+len(t2)+len(gt)+len(t1s)+len(t1c))<5:
            print("there is a problem here! the problem lies in this patient")
            return None, None
        scans_test = [flair[0], t1s[0], t1c[0], t2[0], gt[0]]
        # read a volume composed of 5 modalities
            # print(scans)
        test_im = []
        for k in range(len(scans_test)):
            img = np.array(nib.load(scans_test[k]).dataobj)
            test_im.append(img)

        
        test_im=np.array(test_im).astype(np.float32)
        test_image = test_im[0:4]
        gt=test_im[-1]
        gt[gt==4]=3
        gt = np.transpose(gt, (2, 1, 0))

        #normalize each slice following the same scheme used for training
        test_image = np.transpose(test_image, (0, 3, 2, 1))
        test_image=self.norm_slices(test_image)
        # print(test_image.shape)
        
        #transform the data to channels_last keras format
        test_image = test_image.swapaxes(0,1)
        # print(test_image.shape, 'after swapping axes')
        test_image=np.transpose(test_image,(0,2,3,1))
        # print(test_image.shape, 'after transposing')

        if show:
            verbose=1
        else:
            verbose=0
        # predict classes of each pixel based on the model
        prediction = self.model.predict(test_image,batch_size=self.batch_size_test,verbose=verbose)   
        prediction = np.argmax(prediction, axis=-1)
        prediction=prediction.astype(np.uint8)
        #reconstruct the initial target values .i.e. 0,1,2,4 for prediction and ground truth
        prediction[prediction==3]=4
        gt[gt==3]=4
        
        return np.array(prediction),np.array(gt)

    
    '''
    computes the evaluation metrics on the segmented volume
    input: (1) str 'filepath_image': filepath to test image for segmentation, including file extension
     (2) bool 'save': whether to save to disk or not
     (3) bool 'show': if true, prints the evaluation metrics
    output: np array of all evaluation metrics
    '''
    def evaluate_segmented_volume(self, filepath_image, save, show, save_path):
        predicted_images, gt = self.predict_volume(filepath_image, show)
        if predicted_images is None:
            return None

        if save:
            np.save('/Users/kaitlinylim/Documents/tumorproj/predictions/path_all_train/{}.npy'.format(save_path), predicted_images)
            # tmp = sitk.GetImageFromArray(predicted_images)
            # sitk.WriteImage(tmp,'/Users/kaitlinylim/Documents/tumorproj/predictions/{}.nii.gz'.format(save_path))

        # compute the evaluation metrics
        Dice_complete = DSC_whole(predicted_images, gt)
        Dice_enhancing = DSC_en(predicted_images, gt)
        Dice_core = DSC_core(predicted_images, gt)

        Sensitivity_whole = sensitivity_whole(predicted_images, gt)
        Sensitivity_en = sensitivity_en(predicted_images, gt)
        Sensitivity_core = sensitivity_core(predicted_images, gt)

        Specificity_whole = specificity_whole(predicted_images, gt)
        Specificity_en = specificity_en(predicted_images, gt)
        Specificity_core = specificity_core(predicted_images, gt)

        Hausdorff_whole = hausdorff_whole(predicted_images, gt)
        Hausdorff_en = hausdorff_en(predicted_images, gt)
        Hausdorff_core = hausdorff_core(predicted_images, gt)

        if show:
            print("************************************************************")
            print("Dice complete tumor score : {:0.4f}".format(Dice_complete))
            print("Dice core tumor score (tt sauf vert): {:0.4f}".format(Dice_core))
            print("Dice enhancing tumor score (jaune):{:0.4f} ".format(Dice_enhancing))
            print("**********************************************")
            print("Sensitivity complete tumor score : {:0.4f}".format(Sensitivity_whole))
            print("Sensitivity core tumor score (tt sauf vert): {:0.4f}".format(Sensitivity_core))
            print("Sensitivity enhancing tumor score (jaune):{:0.4f} ".format(Sensitivity_en))
            print("***********************************************")
            print("Specificity complete tumor score : {:0.4f}".format(Specificity_whole))
            print("Specificity core tumor score (tt sauf vert): {:0.4f}".format(Specificity_core))
            print("Specificity enhancing tumor score (jaune):{:0.4f} ".format(Specificity_en))
            print("***********************************************")
            print("Hausdorff complete tumor score : {:0.4f}".format(Hausdorff_whole))
            print("Hausdorff core tumor score (tt sauf vert): {:0.4f}".format(Hausdorff_core))
            print("Hausdorff enhancing tumor score (jaune):{:0.4f} ".format(Hausdorff_en))
            print("***************************************************************\n\n")

        return np.array((Dice_complete, Dice_core, Dice_enhancing,
                    Sensitivity_whole, Sensitivity_core, Sensitivity_en,
                    Specificity_whole, Specificity_core, Specificity_en,
                    Hausdorff_whole, Hausdorff_en, Hausdorff_en))
  

    '''
    returns an np array of predicted volumes
    '''
    def predict_multiple_volumes(self, filepath_volumes, save, show):
        results, ids = [], []
        for patient in filepath_volumes:
            tmp1 = patient.split('/')
            print("volume id:", tmp1[-2] + '/' + tmp1[-1])
            # might need to change save_path
            tmp=self.evaluate_segmented_volume(patient,save=save,show=show,save_path=os.path.basename(patient))
            if tmp is None:
                continue
            # save the results of each volume
            results.append(tmp)
            # save each id for later use
            ids.append(str(tmp1[-2] + '/' + tmp1[-1]))

        res = np.array(results)
        print("mean:", np.mean(res, axis = 0))
        print("std:", np.std(res, axis = 0))
        print("median:", np.median(res, axis = 0))
        print("25 quartile:", np.percentile(res, 25, axis = 0))
        print("75 quartile:", np.percentile(res, 75, axis = 0))
        print("max:", np.max(res, axis = 0))
        print("min:", np.min(res, axis = 0))

        # might need to change
        np.savetxt("/Users/kaitlinylim/Documents/tumorproj/predictions/path_all_train/results.out", res)
        np.savetxt("/Users/kaitlinylim/Documents/tumorproj/predictions/path_all_train/volumes_id.out", ids, fmt='%s')
        return res
  

    '''
    normalizes each slice, excluding gt
    subtracts mean and div by std dev for each slice
    clips top and bottom one percent of pixel intensities
    '''
    def norm_slices(self, slice_not):
        normed_slices = np.zeros((4, 155, 240, 240))
        for slice_ix in range(4):
            normed_slices[slice_ix] = slice_not[slice_ix]
            for mode_ix in range(155): #change this back to 155 if it doesn't work
                normed_slices[slice_ix][mode_ix] = self._normalize(slice_not[slice_ix][mode_ix])
        return normed_slices

    
    def _normalize(self, slice):
        b = np.percentile(slice, 99)
        t = np.percentile(slice, 1)
        slice = np.clip(slice, t, b)
        image_nonzero = slice[np.nonzero(slice)]

        if np.std(slice) == 0 or np.std(image_nonzero) == 0:
            return slice
        else:
            tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
            tmp[tmp == tmp.min()] = -9
        return tmp

## `Pipeline` Execution
Now, we will actually use the `Pipeline`, `Training`, and `Prediction` classes. We will begin with the patch extraction with the `Pipeline` class. Note that as we convert the data in `Patches` and `Y_labels` to fit the `channels_last` format in Keras, we will need to one-hot encode the targets.

In [27]:
# paths to the BraTS 2018 dataset
path_HGG = glob('/Users/kaitlinylim/Documents/tumorproj/images/BraTS 2018 Training/HGG/**')
path_LGG = glob('/Users/kaitlinylim/Documents/tumorproj/images/BraTS 2018 Training/LGG/**')

# shuffle the datasets
np.random.seed(2022)
np.random.shuffle(path_HGG)
np.random.shuffle(path_LGG)

We will divide the data that we have into 80 percent training data and 20 percent testing.

In [28]:
train_val_HGG = (int)(len(path_HGG) * 0.8)
path_HGG_train = path_HGG[0:train_val_HGG]
path_HGG_test = path_HGG[train_val_HGG:]

train_val_LGG = (int)(len(path_LGG) * 0.8)
path_LGG_train = path_LGG[0:train_val_LGG]
path_LGG_test = path_LGG[train_val_LGG:]

path_all_train = path_HGG_train + path_LGG_train
path_all_test = path_HGG_test + path_LGG_test

print("total number of training images is", len(path_all_train))

total number of training images is 228


Now, we will create the pipeline that extracts patches from the MRI images.

In [ ]:
pipeline = Pipeline(list_train = path_all_train)

Because we are only using MRI scans from patients that have all five modalities, we actually end up eliminating a lot of the patients. First, we will create the necessary variables that we need for the `sample_patches_randomly` method.

In [28]:
np.random.seed(1555)
# might need to change this
start = 0
end = 57
# set the total number of patches
# this formula extracts approximately 3 patches per slice
num_patches = 146 * (end - start) * 3
# define the size of a patch
h = 128
w = 128
d = 4

And now, we will actually sample the patches.

In [29]:
X_patches, Y_labels = pipeline.sample_patches_randomly(num_patches, d, h, w)

begin sample_patches_randomly function
sampling patches


In [30]:
X_patches.shape

(24966, 4, 128, 128)

We will transform the data to `channels_last` Keras format.

In [32]:
X_patches = np.transpose(X_patches, (0, 2, 3, 1)).astype(np.float32)

Because this dataset only has four labels, we will have four classes when we one-hot encode the targets and then transform `y` to one-hot encode the targets.

In [33]:
Y_labels[Y_labels == 4] = 3

shp = Y_labels.shape[0]
Y_labels = Y_labels.reshape(-1)
Y_labels = np_utils.to_categorical(Y_labels).astype(np.uint8)
Y_labels = Y_labels.reshape(shp, h, w, 4)

Now we will shuffle the entire dataset.

In [34]:
shuffle = list(zip(X_patches, Y_labels))
np.random.seed(180)
np.random.shuffle(shuffle)
X_patches = np.array([shuffle[i][0] for i in range(len(shuffle))])
Y_labels = np.array([shuffle[i][1] for i in range(len(shuffle))])
del shuffle

In [35]:
print("size of the patches:", X_patches.shape)
print("size of their corresponding targets:", Y_labels.shape)

size of the patches: (24966, 128, 128, 4)
size of their corresponding targets: (24966, 128, 128, 4)


And then we will save the patches just in case.

In [ ]:
np.save('/Users/kaitlinylim/Documents/tumorproj/numpy/X_patches.npy', X_patches)
np.save('/Users/kaitlinylim/Documents/tumorproj/numpy/Y_labels.npy', Y_labels)

## `Training` Execution
Now to actually train the model, we will use the `Training` class we previously defined.

In [29]:
X_patches = np.load('/Users/kaitlinylim/Documents/tumorproj/numpy/X_patches.npy')
Y_labels = np.load('/Users/kaitlinylim/Documents/tumorproj/numpy/Y_labels.npy')

We will split the training data further into training data and validation data, 80% and 20% respectively.

In [30]:
# split the training data further into training data and validation data
div = int(X_patches.shape[0] * 0.8)
X_training = X_patches[0:div]
X_validation = X_patches[div:]
Y_training = Y_labels[0:div]
Y_validation = Y_labels[div:]

We will next compile the model using the `Training` class, with a `batch_size` of 4 and 2 epochs.

In [91]:
brain_seg_1 = Training(batch_size = 4, nb_epoch = 2)

u-net cnn compiled!


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


Let's see how many trainable parameters there are in the CNN.

In [92]:
print("number of trainable parameters:", brain_seg_1.model.count_params())
print(brain_seg_1.model.summary())

number of trainable parameters: 10159748
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 128, 128, 4)  0                                            
__________________________________________________________________________________________________
gaussian_noise_16 (GaussianNois (None, 128, 128, 4)  0           input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, 128, 128, 64) 1088        gaussian_noise_16[0][0]          
__________________________________________________________________________________________________
batch_normalization_226 (BatchN (None, 128, 128, 64) 256         conv2d_376[0][0]                 
____________________________________________________________________

Now, we are using the patches that we have just created to train the model.

In [ ]:
brain_seg_1.fit_unet(X_training, Y_training, X_validation, Y_validation)

After successfully training the model, we will now save it.

In [92]:
brain_seg_1.save_model('seg_2_epochs')

model saved


We will also create another model with four epochs instead of two. But instead of creating and training another model from scratch, we will instead load the model that we just saved and "resume" training.

In [122]:
model_to_load = 'seg_2_epochs'
brain_seg_2 = Training(batch_size = 4, nb_epoch = 2, load_model_resume_training = model_to_load)

Loading model seg_2_epochs
model loaded.
model compiled
pre-trained model loaded!


As we did previously, we will train and save the model.

In [ ]:
brain_seg_2.fit_unet(X_training, Y_training, X_validation, Y_validation)
brain_seg_2.save_model('seg_4_epochs')

## `Prediction` Execution
Now we will use the newly created model for predictions! We will first create an object in the `Prediction` class, which basically just loads a fresh U-Net model on which the weights will be loaded for predictions.

In [173]:
model_to_load = '/Users/kaitlinylim/Documents/tumorproj/weights/seg_2_epochs.hdf5'
brain_seg_pred = Prediction(batch_size_test = 2, load_model_path = model_to_load)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


u-net cnn model compiled



We will then predict each volume and then save them to a Numpy array. We will first use the testing data.

In [123]:
final_vol_stats = brain_seg_pred.predict_multiple_volumes(path_all_test, save = True, show = True)

volume id: HGG/Brats18_TCIA02_309_1
there is a problem here! the problem lies in this patient
volume id: HGG/Brats18_TCIA02_321_1
155/155 [==============================] - 287s 2s/step
************************************************************
Dice complete tumor score : 0.8037
Dice core tumor score (tt sauf vert): 0.8337
Dice enhancing tumor score (jaune):0.8119 
**********************************************
Sensitivity complete tumor score : 0.9510
Sensitivity core tumor score (tt sauf vert): 0.8909
Sensitivity enhancing tumor score (jaune):0.8610 
***********************************************
Specificity complete tumor score : 0.9939
Specificity core tumor score (tt sauf vert): 0.9984
Specificity enhancing tumor score (jaune):0.9985 
***********************************************
Hausdorff complete tumor score : 14.5945
Hausdorff core tumor score (tt sauf vert): 14.5945
Hausdorff enhancing tumor score (jaune):6.4031 
***********************************************************

155/155 [==============================] - 274s 2s/step
************************************************************
Dice complete tumor score : 0.4498
Dice core tumor score (tt sauf vert): 0.6123
Dice enhancing tumor score (jaune):0.5868 
**********************************************
Sensitivity complete tumor score : 0.6721
Sensitivity core tumor score (tt sauf vert): 0.6158
Sensitivity enhancing tumor score (jaune):0.5478 
***********************************************
Specificity complete tumor score : 0.9968
Specificity core tumor score (tt sauf vert): 0.9994
Specificity enhancing tumor score (jaune):0.9996 
***********************************************
Hausdorff complete tumor score : 4.4721
Hausdorff core tumor score (tt sauf vert): 5.3852
Hausdorff enhancing tumor score (jaune):4.1231 
***************************************************************


volume id: HGG/Brats18_TCIA05_478_1
there is a problem here! the problem lies in this patient
volume id: HGG/Brats18_TCIA04_3

And because the current amount of testing data would likely not be enough for the training of the classification models, we will also use the CNN to predict the training data.

In [174]:
final_vol_stats_2 = brain_seg_pred.predict_multiple_volumes(path_all_train, save = True, show = True)

volume id: HGG/Brats18_TCIA04_149_1
there is a problem here! the problem lies in this patient
volume id: HGG/Brats18_TCIA06_211_1
155/155 [==============================] - 283s 2s/step
************************************************************
Dice complete tumor score : 0.8569
Dice core tumor score (tt sauf vert): 0.7875
Dice enhancing tumor score (jaune):0.7885 
**********************************************
Sensitivity complete tumor score : 0.8688
Sensitivity core tumor score (tt sauf vert): 0.8795
Sensitivity enhancing tumor score (jaune):0.8338 
***********************************************
Specificity complete tumor score : 0.9972
Specificity core tumor score (tt sauf vert): 0.9969
Specificity enhancing tumor score (jaune):0.9978 
***********************************************
Hausdorff complete tumor score : 10.3441
Hausdorff core tumor score (tt sauf vert): 13.1529
Hausdorff enhancing tumor score (jaune):11.1803 
**********************************************************

155/155 [==============================] - 281s 2s/step
************************************************************
Dice complete tumor score : 0.6323
Dice core tumor score (tt sauf vert): 0.3853
Dice enhancing tumor score (jaune):0.4806 
**********************************************
Sensitivity complete tumor score : 0.7747
Sensitivity core tumor score (tt sauf vert): 0.3871
Sensitivity enhancing tumor score (jaune):0.3794 
***********************************************
Specificity complete tumor score : 0.9984
Specificity core tumor score (tt sauf vert): 0.9997
Specificity enhancing tumor score (jaune):0.9999 
***********************************************
Hausdorff complete tumor score : 6.0828
Hausdorff core tumor score (tt sauf vert): 3.0000
Hausdorff enhancing tumor score (jaune):3.3166 
***************************************************************


volume id: HGG/Brats18_CBICA_AWH_1
there is a problem here! the problem lies in this patient
volume id: HGG/Brats18_CBICA_ASW

155/155 [==============================] - 272s 2s/step
************************************************************
Dice complete tumor score : 0.7401
Dice core tumor score (tt sauf vert): 0.7100
Dice enhancing tumor score (jaune):0.5102 
**********************************************
Sensitivity complete tumor score : 0.9775
Sensitivity core tumor score (tt sauf vert): 0.7363
Sensitivity enhancing tumor score (jaune):0.8788 
***********************************************
Specificity complete tumor score : 0.9904
Specificity core tumor score (tt sauf vert): 0.9984
Specificity enhancing tumor score (jaune):0.9977 
***********************************************
Hausdorff complete tumor score : 16.3095
Hausdorff core tumor score (tt sauf vert): 9.4868
Hausdorff enhancing tumor score (jaune):8.6603 
***************************************************************


volume id: HGG/Brats18_TCIA02_473_1
there is a problem here! the problem lies in this patient
volume id: HGG/Brats18_CBICA_A

155/155 [==============================] - 269s 2s/step
************************************************************
Dice complete tumor score : 0.4667
Dice core tumor score (tt sauf vert): 0.0400
Dice enhancing tumor score (jaune):0.0070 
**********************************************
Sensitivity complete tumor score : 0.3597
Sensitivity core tumor score (tt sauf vert): 0.0215
Sensitivity enhancing tumor score (jaune):0.0037 
***********************************************
Specificity complete tumor score : 0.9995
Specificity core tumor score (tt sauf vert): 0.9999
Specificity enhancing tumor score (jaune):1.0000 
***********************************************
Hausdorff complete tumor score : 4.0000
Hausdorff core tumor score (tt sauf vert): 1.4142
Hausdorff enhancing tumor score (jaune):1.7321 
***************************************************************


volume id: HGG/Brats18_CBICA_AXJ_1
155/155 [==============================] - 269s 2s/step
**********************************

155/155 [==============================] - 268s 2s/step
************************************************************
Dice complete tumor score : 0.6281
Dice core tumor score (tt sauf vert): 0.6019
Dice enhancing tumor score (jaune):0.5666 
**********************************************
Sensitivity complete tumor score : 0.8858
Sensitivity core tumor score (tt sauf vert): 0.7676
Sensitivity enhancing tumor score (jaune):0.7842 
***********************************************
Specificity complete tumor score : 0.9925
Specificity core tumor score (tt sauf vert): 0.9956
Specificity enhancing tumor score (jaune):0.9971 
***********************************************
Hausdorff complete tumor score : 7.0000
Hausdorff core tumor score (tt sauf vert): 8.6023
Hausdorff enhancing tumor score (jaune):7.2111 
***************************************************************


volume id: HGG/Brats18_CBICA_BHM_1
155/155 [==============================] - 270s 2s/step
**********************************

155/155 [==============================] - 270s 2s/step
************************************************************
Dice complete tumor score : 0.7987
Dice core tumor score (tt sauf vert): 0.4493
Dice enhancing tumor score (jaune):0.0000 
**********************************************
Sensitivity complete tumor score : 0.9163
Sensitivity core tumor score (tt sauf vert): 0.3615
Sensitivity enhancing tumor score (jaune):1.0000 
***********************************************
Specificity complete tumor score : 0.9965
Specificity core tumor score (tt sauf vert): 0.9990
Specificity enhancing tumor score (jaune):1.0000 
***********************************************
Hausdorff complete tumor score : 8.5440
Hausdorff core tumor score (tt sauf vert): 7.3485
Hausdorff enhancing tumor score (jaune):2.2361 
***************************************************************


volume id: LGG/Brats18_TCIA10_449_1
155/155 [==============================] - 270s 2s/step
*********************************

155/155 [==============================] - 271s 2s/step
************************************************************
Dice complete tumor score : 0.7746
Dice core tumor score (tt sauf vert): 0.5504
Dice enhancing tumor score (jaune):0.0000 
**********************************************
Sensitivity complete tumor score : 0.8890
Sensitivity core tumor score (tt sauf vert): 0.4686
Sensitivity enhancing tumor score (jaune):1.0000 
***********************************************
Specificity complete tumor score : 0.9975
Specificity core tumor score (tt sauf vert): 0.9991
Specificity enhancing tumor score (jaune):0.9999 
***********************************************
Hausdorff complete tumor score : 4.6904
Hausdorff core tumor score (tt sauf vert): 7.6811
Hausdorff enhancing tumor score (jaune):0.0000 
***************************************************************


volume id: LGG/Brats18_TCIA10_490_1
there is a problem here! the problem lies in this patient
volume id: LGG/Brats18_2013_9_1

Finally, we're finished! We will go through each predicted image and choose the predictions with a complete tumor Dice score of 0.7 or greater for classification. A Dice score of approximately 0.85 is quite good, but just to have a large enough repository of data for training and predicting, we will use predictions with scores of 0.70 or greater.